In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.regularizers import l2
import keras
from keras.layers import LSTM, Dense,Input, Conv2D, Flatten, BatchNormalization

In [8]:
filename="DB_modified.csv"
dataset = pd.read_csv(filename)
dataset
# Q1 = dataset.quantile(0.25)
# Q3 = dataset.quantile(0.75)
# IQR = Q3 - Q1
# # print(IQR)
# # print(dataset.shape)
# dataset = dataset[~((dataset < (Q1 - 1.5 * IQR)) |(dataset > (Q3 + 1.5 * IQR))).any(axis=1)]
# print(dataset.shape)
dataset =np.array(dataset)[1:-1]
dataset=np.asfarray(dataset,float)
dataset.shape

(2158, 8)

In [9]:
X=dataset[:,:4]
print(X)
# X=(X-np.mean(X))/np.std(X)

# X=dataset[:,2]
X[:,0]=(X[:,0]-np.mean(X[:,0]))/np.std(X[:,0])
X[:,1]=(X[:,1]-np.mean(X[:,1]))/np.std(X[:,1])
X[:,2]=(X[:,2]-np.mean(X[:,2]))/np.std(X[:,2])
X[:,3]=(X[:,3]-np.mean(X[:,3]))/np.std(X[:,3])

# X[:,1]=X[:,1]/np.max(X[:,1])
y=dataset[:,4:7]
# Vthin=dataset[:,-2]
# np.concatenate((X, np.expand_dims(Vthin, 1)), 1)
# print(X)
print(y.shape)
# X=np.expand_dims(X, 1)
y,X

[[0.0e+00 2.0e+16 1.5e-02 1.7e-01]
 [0.0e+00 2.0e+16 1.5e-02 1.9e-01]
 [0.0e+00 2.0e+16 1.5e-02 2.1e-01]
 ...
 [1.0e+00 1.0e+19 2.9e-02 2.5e-01]
 [1.0e+00 1.0e+19 2.9e-02 2.7e-01]
 [1.0e+00 1.0e+19 2.9e-02 2.9e-01]]
(2158, 3)


(array([[ 38.9934 ,  82.224  ,  -0.716  ],
        [ 58.829  ,  90.156  ,  -0.977  ],
        [ 82.97375,  97.828  ,  -1.239  ],
        ...,
        [404.002  , 101.472  ,  -5.41   ],
        [444.592  , 106.572  ,  -5.935  ],
        [483.73   , 111.408  ,  -6.464  ]]),
 array([[-1.        , -0.69961252, -1.5284699 , -1.16264952],
        [-1.        , -0.69961252, -1.5284699 , -0.77509968],
        [-1.        , -0.69961252, -1.5284699 , -0.38754984],
        ...,
        [ 1.        ,  2.47781879,  1.5284699 ,  0.38754984],
        [ 1.        ,  2.47781879,  1.5284699 ,  0.77509968],
        [ 1.        ,  2.47781879,  1.5284699 ,  1.16264952]]))

In [10]:
dfgan = pd.read_csv("fromGANwo-ve.csv")
# Q1 = dfgan.quantile(0.25)
# Q3 = dfgan.quantile(0.75)
# IQR = Q3 - Q1
# print(IQR)
# print(dfgan.shape)
# dfgan = dfgan[~((dfgan < (Q1 - 1.5 * IQR)) |(dfgan > (Q3 + 1.5 * IQR))).any(axis=1)]
# print(dfgan.shape)
dfgan =np.array(dfgan)

dfgan=np.asfarray(dfgan,float)
xGAN=dfgan[:,0:4]
print(xGAN)
xGAN[:,0]=(xGAN[:,0]-np.mean(xGAN[:,0]))/np.std(xGAN[:,0])
xGAN[:,1]=(xGAN[:,1]-np.mean(xGAN[:,1]))/np.std(xGAN[:,1])
xGAN[:,2]=(xGAN[:,2]-np.mean(xGAN[:,2]))/np.std(xGAN[:,2])
xGAN[:,3]=(xGAN[:,3]-np.mean(xGAN[:,3]))/np.std(xGAN[:,3])
yGAN=dfgan[:,4:7]
print(xGAN)

[[-0.6972414   1.4884295   0.51256835  0.5147029 ]
 [-1.0893219   1.1557536  -0.9450898   1.4321591 ]
 [ 0.9408671  -0.574       0.32613173  0.571942  ]
 ...
 [ 0.86686724 -0.25266343  0.79319525  1.412123  ]
 [-0.9377408  -0.06915441 -1.0210751  -1.4790384 ]
 [-0.735737    1.30408     0.27639705 -1.70117   ]]
[[-0.85502875  1.10179663  0.58732684  0.50957876]
 [-1.29633887  0.79686945 -0.91004294  1.46655927]
 [ 0.98876059 -0.78860423  0.39581105  0.56928374]
 ...
 [ 0.90546931 -0.49407056  0.87559899  1.44566001]
 [-1.12572525 -0.32586815 -0.98809834 -1.57005355]
 [-0.89835786  0.93282385  0.34472142 -1.80175465]]


In [11]:
xtotal=np.vstack((X, xGAN))
ytotal=np.vstack((y, yGAN))
print(xtotal)
ytotal.shape

[[-1.         -0.69961252 -1.5284699  -1.16264952]
 [-1.         -0.69961252 -1.5284699  -0.77509968]
 [-1.         -0.69961252 -1.5284699  -0.38754984]
 ...
 [ 0.90546931 -0.49407056  0.87559899  1.44566001]
 [-1.12572525 -0.32586815 -0.98809834 -1.57005355]
 [-0.89835786  0.93282385  0.34472142 -1.80175465]]


(3111, 3)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(xtotal, ytotal, test_size=0.15, random_state=42, shuffle=True)
print(x_train.shape)
x_train, x_cv, y_train, y_cv = train_test_split(x_train, y_train, test_size=0.15, random_state=42, shuffle=True)
print(y_train.shape)

(2644, 4)
(2247, 3)


In [13]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_cv.shape)
print(y_cv.shape)

(2247, 4)
(2247, 3)
(467, 4)
(467, 3)
(397, 4)
(397, 3)


In [14]:
# #SVR
# from sklearn.svm import SVR
# regressor = SVR(kernel='poly', degree=20)
# regressor.fit(x_train,y_train)
# for i in range(50):
#     y_pred = regressor.predict([x_test[i]])
#     print(y_pred-y_test[i])

In [18]:
x_traindim =np.expand_dims(x_train, 1)
y_traindim =np.expand_dims(y_train, 1)
x_cvdim =np.expand_dims(x_cv, 1)
y_cvdim =np.expand_dims(y_cv, 1)
x_testdim =np.expand_dims(x_test, 1)
y_testdim =np.expand_dims(y_test, 1)
print(x_traindim)
print(x_traindim.shape)

[[[ 1.03335528  1.77847337 -0.30606683 -1.10101673]]

 [[ 0.86371599 -0.36184192 -0.95099599 -0.29728895]]

 [[-1.         -0.69961252 -1.5284699  -0.77509968]]

 ...

 [[-1.         -0.06922033 -0.65505853 -1.55019935]]

 [[-1.         -0.4512762   0.65505853  1.55019935]]

 [[ 1.         -0.67414212 -0.65505853  0.77509968]]]
(2247, 1, 4)


In [26]:
for m in range(3):
    for k in range(3):
        for i in range(3):
                c1=32*2**m
                c2=32*2**k
                c3=32*2**i
#                 cnt=cnt+1
                if(not((c1==32 and c2==32 and c3==32)or (c1==32 and c2==32 and c3==64)or(c1==32 and c2==64 and c3==32)or(c1==64 and c2==32 and c3==32)or (c1==32 and c2==32 and c3==128))):
                    filename='L'+str(c1)+'L'+str(c2)+'L'+str(c3)
                    
                    modelfilepath="E:\\Simulations\\Silvaco\\\\2D HEMT\\EDL\\DATA for modeling\\modelLSTMvar_2D64_D3\\"+filename
                    
                    model = Sequential()
                    model.add(LSTM(c1, return_sequences=True, input_shape=(x_traindim[1].shape)))
                    model.add(LSTM(c2, return_sequences=True))
                    model.add(LSTM(c3, return_sequences=False))
                    # model.add(LSTM(16, return_sequences=True))
                    # model.add(LSTM(16, return_sequences=True))
                    # model.add(LSTM(16, return_sequences=True))
                    # model.add(LSTM(32, return_sequences=True))
                    # model.add(LSTM(16))

                    model.add(Dense(64, activation='relu'))
                    model.add(Dense(64, activation='relu'))

                    # model.add(Dense(64, activation='relu'))
                    # # # model.add(BatchNormalization())
                    # model.add(Dense(128, activation='relu'))
                    # model.add(Dense(64, activation='relu'))
                    # model.add(Dense(64, activation='relu'))
                    # model.add(Dense(64, activation='relu'))
                    # model.add(BatchNormalization())
                    # model.add(Dense(64, activation='relu'))
                    # model.add(Dense(128, activation='relu'))
                    # model.add(Dense(512, activation='relu'))
                    # model.add(Dense(128, activation='relu'))
                    # model.add(Dense(64, activation='relu'))
                    # model.add(Dense(128, activation='relu'))
                    # model.add(Dense(64, activation='relu'))
                    # model.add(Dense(64, activation='relu'))
                    # model.add(Dropout(0.5))
                    # model.add(Dense(64, activation='relu'))
                    # model.add(Dense(64, activation='relu'))
                    model.add(Dense(3, activation='linear'))
#                     opt=keras.optimizers.adam(0.001)
                    model.compile(loss='mean_squared_error', optimizer="adam", metrics=['MSE'])
                    model.summary()
                    filepath="E:\\Simulations\\Silvaco\\\\2D HEMT\\EDL\\DATA for modeling\\modelLSTMvar_2D64_D3\\latestmodel.hdf5"
                    checkpointer = ModelCheckpoint( monitor='val_loss',filepath=filepath, verbose=1, save_best_only=True)
                    history3=model.fit(x_traindim,y_train, validation_data=(x_cvdim, y_cv), epochs=1500, batch_size=16, callbacks=[checkpointer])
                    minloss=round(min(history3.history['val_loss']), 1) 
                    modelfilepath="E:\\Simulations\\Silvaco\\\\2D HEMT\\EDL\\DATA for modeling\\modelLSTMvar_2D64_D3\\"+filename+'_'+str(minloss)
                    df=pd.DataFrame(history3.history)
                    model.save(modelfilepath+'.h5')
                    df.to_csv(modelfilepath+'.csv')
                    


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1, 32)             4736      
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 64)             24832     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_15 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_16 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 195       
Total params: 71,107
Trainable params: 71,107
Non-trainable params: 0
__________________________________________________

KeyboardInterrupt: 

In [37]:
#model used for only actualdatabut now being testd for GAN too
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(x_train[1].shape)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['MSE'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_31 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_32 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_33 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_34 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_35 (Dense)             (None, 3)                 195       
Total params: 8,739
Trainable params: 8,739
Non-trainable params: 0
____________________________________________________

In [38]:
filepath="E:\\Study\\Ph.D\\Publication\\EDL\\ML modelling related files"
filename=filepath+"3D32_2d64_relu_ALL.h5"
checkpointer = ModelCheckpoint( monitor='val_loss',filepath=filename, verbose=1, save_best_only=True)
history3=model.fit(x_train,y_train, validation_data=(x_cv, y_cv), epochs=1500, batch_size=16, callbacks=[checkpointer])

Epoch 1/1500
141/141 [==============================] - 4s 12ms/step - loss: 22664.3659 - MSE: 22664.3659 - val_loss: 1613.9600 - val_MSE: 1613.9600

Epoch 00001: val_loss improved from inf to 1613.95996, saving model to E:\Study\Ph.D\Publication\EDL\ML modelling related files3D32_2d64_relu_ALL.h5
Epoch 2/1500
141/141 [==============================] - 1s 8ms/step - loss: 1580.9779 - MSE: 1580.9779 - val_loss: 1180.8993 - val_MSE: 1180.8993

Epoch 00002: val_loss improved from 1613.95996 to 1180.89929, saving model to E:\Study\Ph.D\Publication\EDL\ML modelling related files3D32_2d64_relu_ALL.h5
Epoch 3/1500
141/141 [==============================] - 1s 8ms/step - loss: 1066.4182 - MSE: 1066.4182 - val_loss: 1013.6926 - val_MSE: 1013.6926

Epoch 00003: val_loss improved from 1180.89929 to 1013.69263, saving model to E:\Study\Ph.D\Publication\EDL\ML modelling related files3D32_2d64_relu_ALL.h5
Epoch 4/1500
141/141 [==============================] - 1s 9ms/step - loss: 1086.4560 - MSE: 10

Epoch 34/1500
141/141 [==============================] - 1s 8ms/step - loss: 349.7796 - MSE: 349.7796 - val_loss: 349.4695 - val_MSE: 349.4695

Epoch 00034: val_loss did not improve from 346.22879
Epoch 35/1500
141/141 [==============================] - 1s 8ms/step - loss: 399.8688 - MSE: 399.8688 - val_loss: 401.5255 - val_MSE: 401.5255

Epoch 00035: val_loss did not improve from 346.22879
Epoch 36/1500
141/141 [==============================] - 1s 8ms/step - loss: 356.8862 - MSE: 356.8862 - val_loss: 421.1308 - val_MSE: 421.1308

Epoch 00036: val_loss did not improve from 346.22879
Epoch 37/1500
141/141 [==============================] - 1s 8ms/step - loss: 374.5532 - MSE: 374.5532 - val_loss: 436.9595 - val_MSE: 436.9595

Epoch 00037: val_loss did not improve from 346.22879
Epoch 38/1500
141/141 [==============================] - 1s 8ms/step - loss: 424.6053 - MSE: 424.6053 - val_loss: 352.9364 - val_MSE: 352.9364

Epoch 00038: val_loss did not improve from 346.22879
Epoch 39/1500
1

141/141 [==============================] - 1s 8ms/step - loss: 319.0614 - MSE: 319.0614 - val_loss: 362.1568 - val_MSE: 362.1568

Epoch 00072: val_loss did not improve from 314.78421
Epoch 73/1500
141/141 [==============================] - 1s 8ms/step - loss: 282.1558 - MSE: 282.1558 - val_loss: 347.0042 - val_MSE: 347.0042

Epoch 00073: val_loss did not improve from 314.78421
Epoch 74/1500
141/141 [==============================] - 1s 8ms/step - loss: 295.3218 - MSE: 295.3218 - val_loss: 405.0544 - val_MSE: 405.0544

Epoch 00074: val_loss did not improve from 314.78421
Epoch 75/1500
141/141 [==============================] - 1s 8ms/step - loss: 296.2674 - MSE: 296.2674 - val_loss: 357.6578 - val_MSE: 357.6578

Epoch 00075: val_loss did not improve from 314.78421
Epoch 76/1500
141/141 [==============================] - 1s 8ms/step - loss: 271.0011 - MSE: 271.0011 - val_loss: 322.3716 - val_MSE: 322.3716

Epoch 00076: val_loss did not improve from 314.78421
Epoch 77/1500
141/141 [======


Epoch 00109: val_loss did not improve from 264.19922
Epoch 110/1500
141/141 [==============================] - 1s 9ms/step - loss: 243.4889 - MSE: 243.4889 - val_loss: 266.5740 - val_MSE: 266.5740

Epoch 00110: val_loss did not improve from 264.19922
Epoch 111/1500
141/141 [==============================] - 1s 8ms/step - loss: 238.5181 - MSE: 238.5181 - val_loss: 319.9163 - val_MSE: 319.9163

Epoch 00111: val_loss did not improve from 264.19922
Epoch 112/1500
141/141 [==============================] - 1s 8ms/step - loss: 324.3174 - MSE: 324.3174 - val_loss: 316.0370 - val_MSE: 316.0370

Epoch 00112: val_loss did not improve from 264.19922
Epoch 113/1500
141/141 [==============================] - 1s 8ms/step - loss: 191.5289 - MSE: 191.5289 - val_loss: 306.6042 - val_MSE: 306.6042

Epoch 00113: val_loss did not improve from 264.19922
Epoch 114/1500
141/141 [==============================] - 1s 8ms/step - loss: 226.3404 - MSE: 226.3404 - val_loss: 316.1476 - val_MSE: 316.1476

Epoch 001


Epoch 00147: val_loss did not improve from 222.63222
Epoch 148/1500
141/141 [==============================] - 1s 8ms/step - loss: 224.3207 - MSE: 224.3207 - val_loss: 261.4983 - val_MSE: 261.4983

Epoch 00148: val_loss did not improve from 222.63222
Epoch 149/1500
141/141 [==============================] - 1s 8ms/step - loss: 184.3626 - MSE: 184.3626 - val_loss: 241.5697 - val_MSE: 241.5697

Epoch 00149: val_loss did not improve from 222.63222
Epoch 150/1500
141/141 [==============================] - ETA: 0s - loss: 250.7401 - MSE: 250.740 - 1s 8ms/step - loss: 250.3727 - MSE: 250.3727 - val_loss: 280.6692 - val_MSE: 280.6692

Epoch 00150: val_loss did not improve from 222.63222
Epoch 151/1500
141/141 [==============================] - 1s 8ms/step - loss: 194.7611 - MSE: 194.7611 - val_loss: 240.8976 - val_MSE: 240.8976

Epoch 00151: val_loss did not improve from 222.63222
Epoch 152/1500
141/141 [==============================] - 1s 8ms/step - loss: 170.1446 - MSE: 170.1446 - val_los

...

141/141 [==============================] - 1s 7ms/step - loss: 71.1741 - MSE: 71.1741 - val_loss: 153.1491 - val_MSE: 153.1491

Epoch 01306: val_loss did not improve from 91.61790
Epoch 1307/1500
141/141 [==============================] - 1s 7ms/step - loss: 77.6532 - MSE: 77.6532 - val_loss: 151.1482 - val_MSE: 151.1482

Epoch 01307: val_loss did not improve from 91.61790
Epoch 1308/1500
141/141 [==============================] - 1s 8ms/step - loss: 47.6519 - MSE: 47.6519 - val_loss: 156.2716 - val_MSE: 156.2716

Epoch 01308: val_loss did not improve from 91.61790
Epoch 1309/1500
141/141 [==============================] - 1s 7ms/step - loss: 90.6320 - MSE: 90.6320 - val_loss: 167.9751 - val_MSE: 167.9751

Epoch 01309: val_loss did not improve from 91.61790
Epoch 1310/1500
141/141 [==============================] - 1s 7ms/step - loss: 96.7568 - MSE: 96.7568 - val_loss: 163.7869 - val_MSE: 163.7869

Epoch 01310: val_loss did not improve from 91.61790
Epoch 1311/1500
141/141 [===========

141/141 [==============================] - 1s 9ms/step - loss: 62.8353 - MSE: 62.8353 - val_loss: 178.7778 - val_MSE: 178.7778

Epoch 01348: val_loss did not improve from 91.61790
Epoch 1349/1500
141/141 [==============================] - 1s 8ms/step - loss: 64.7544 - MSE: 64.7544 - val_loss: 181.2210 - val_MSE: 181.2210

Epoch 01349: val_loss did not improve from 91.61790
Epoch 1350/1500
141/141 [==============================] - 1s 8ms/step - loss: 48.8835 - MSE: 48.8835 - val_loss: 159.1306 - val_MSE: 159.1306

Epoch 01350: val_loss did not improve from 91.61790
Epoch 1351/1500
141/141 [==============================] - 1s 8ms/step - loss: 59.1912 - MSE: 59.1912 - val_loss: 166.3796 - val_MSE: 166.3796

Epoch 01351: val_loss did not improve from 91.61790
Epoch 1352/1500
141/141 [==============================] - 1s 7ms/step - loss: 53.1827 - MSE: 53.1827 - val_loss: 175.6657 - val_MSE: 175.6657

Epoch 01352: val_loss did not improve from 91.61790
Epoch 1353/1500
141/141 [===========

141/141 [==============================] - 1s 8ms/step - loss: 64.0422 - MSE: 64.0422 - val_loss: 160.6931 - val_MSE: 160.6931

Epoch 01390: val_loss did not improve from 91.61790
Epoch 1391/1500
141/141 [==============================] - 1s 7ms/step - loss: 35.4137 - MSE: 35.4137 - val_loss: 150.7921 - val_MSE: 150.7921

Epoch 01391: val_loss did not improve from 91.61790
Epoch 1392/1500
141/141 [==============================] - 1s 8ms/step - loss: 67.3190 - MSE: 67.3190 - val_loss: 163.2545 - val_MSE: 163.2545

Epoch 01392: val_loss did not improve from 91.61790
Epoch 1393/1500
141/141 [==============================] - 1s 8ms/step - loss: 57.7041 - MSE: 57.7041 - val_loss: 210.5717 - val_MSE: 210.5717

Epoch 01393: val_loss did not improve from 91.61790
Epoch 1394/1500
141/141 [==============================] - 1s 8ms/step - loss: 56.2893 - MSE: 56.2893 - val_loss: 190.8042 - val_MSE: 190.8042

Epoch 01394: val_loss did not improve from 91.61790
Epoch 1395/1500
141/141 [===========

141/141 [==============================] - 1s 7ms/step - loss: 118.1139 - MSE: 118.1139 - val_loss: 182.0285 - val_MSE: 182.0285

Epoch 01432: val_loss did not improve from 91.61790
Epoch 1433/1500
141/141 [==============================] - 1s 8ms/step - loss: 64.3311 - MSE: 64.3311 - val_loss: 167.8849 - val_MSE: 167.8849

Epoch 01433: val_loss did not improve from 91.61790
Epoch 1434/1500
141/141 [==============================] - 1s 8ms/step - loss: 77.5659 - MSE: 77.5659 - val_loss: 181.3445 - val_MSE: 181.3445

Epoch 01434: val_loss did not improve from 91.61790
Epoch 1435/1500
141/141 [==============================] - 1s 8ms/step - loss: 51.1660 - MSE: 51.1660 - val_loss: 206.0607 - val_MSE: 206.0607

Epoch 01435: val_loss did not improve from 91.61790
Epoch 1436/1500
141/141 [==============================] - 1s 8ms/step - loss: 77.2554 - MSE: 77.2554 - val_loss: 160.4703 - val_MSE: 160.4703

Epoch 01436: val_loss did not improve from 91.61790
Epoch 1437/1500
141/141 [=========

141/141 [==============================] - 1s 8ms/step - loss: 53.0119 - MSE: 53.0119 - val_loss: 158.8151 - val_MSE: 158.8151

Epoch 01474: val_loss did not improve from 91.61790
Epoch 1475/1500
141/141 [==============================] - 1s 8ms/step - loss: 49.5339 - MSE: 49.5339 - val_loss: 151.4277 - val_MSE: 151.4277

Epoch 01475: val_loss did not improve from 91.61790
Epoch 1476/1500
141/141 [==============================] - 1s 8ms/step - loss: 51.4687 - MSE: 51.4687 - val_loss: 167.8572 - val_MSE: 167.8572

Epoch 01476: val_loss did not improve from 91.61790
Epoch 1477/1500
141/141 [==============================] - 1s 8ms/step - loss: 73.4298 - MSE: 73.4298 - val_loss: 140.2355 - val_MSE: 140.2355

Epoch 01477: val_loss did not improve from 91.61790
Epoch 1478/1500
141/141 [==============================] - 1s 8ms/step - loss: 39.5808 - MSE: 39.5808 - val_loss: 142.8461 - val_MSE: 142.8461

Epoch 01478: val_loss did not improve from 91.61790
Epoch 1479/1500
141/141 [===========

In [36]:
minloss=round(min(history3.history['val_loss']), 1) 
modelfilepath = filepath +  '\\history_loss_relu'
df=pd.DataFrame(history3.history)
# model.save(modelfilepath+'.h5')
df.to_csv(modelfilepath+'.csv')

In [70]:
model = Sequential()
model.add(LSTM(64, return_sequences=False, input_shape=(x_traindim[1].shape)))
# model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='linear'))
opt=keras.optimizers.adam(0.001)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['MSE'])
model.summary()

ValueError: Input 0 is incompatible with layer lstm_53: expected ndim=3, found ndim=2

In [74]:
filepath="E:\\Simulations\\Silvaco\\\\2D HEMT\\EDL\\DATA for modeling\\models_history\\FOM_MSE_withGANdata_noDropoutused26july.hdf5"
checkpointer = ModelCheckpoint( monitor='val_loss',filepath=filepath, verbose=1, save_best_only=True)
history3=model.fit(x_traindim,y_train, validation_data=(x_cvdim, y_cv), epochs=10, batch_size=16, callbacks=[checkpointer])


Train on 2247 samples, validate on 397 samples
Epoch 1/10
2247/2247 [==============================] - 22s 10ms/step - loss: 15482.2082 - mean_squared_error: 15482.2082 - val_loss: 1121.5093 - val_mean_squared_error: 1121.5093

Epoch 00001: val_loss improved from inf to 1121.50926, saving model to E:\Simulations\Silvaco\\2D HEMT\EDL\DATA for modeling\models_history\FOM_MSE_withGANdata_noDropoutused26july.hdf5
Epoch 2/10
2247/2247 [==============================] - 4s 2ms/step - loss: 1105.7519 - mean_squared_error: 1105.7519 - val_loss: 943.9846 - val_mean_squared_error: 943.9846

Epoch 00002: val_loss improved from 1121.50926 to 943.98458, saving model to E:\Simulations\Silvaco\\2D HEMT\EDL\DATA for modeling\models_history\FOM_MSE_withGANdata_noDropoutused26july.hdf5
Epoch 3/10
2247/2247 [==============================] - 4s 2ms/step - loss: 983.5061 - mean_squared_error: 983.5061 - val_loss: 906.2233 - val_mean_squared_error: 906.2233

Epoch 00003: val_loss improved from 943.98458 to

In [ ]:
xpred=[]
act=[]
m=0
for i in range(467):
    r=model.predict(np.expand_dims(x_testdim[i,:], axis=0))
#     pred.append(r[0][0])
#     act.append(y_test[i])
#     print((r[0]-y_test[i])/y_test[i], r[0][0], y_test[i] )
#     print('predicted: ', y_test[i],' actual: ', r, 'diff:', (r[0]-y_test[i])/y_test[i]*100)

    if (np.max((r[0]-y_test[i])/y_test[i]*100)>10):
        print('predicted: ', y_test[i],' actual: ', r, 'diff:', (r[0]-y_test[i])/y_test[i]*100)
        m=m+1
m

In [84]:
minloss=round(min(history3.history['val_loss']), 1)
minloss

748.5

In [ ]:
import pandas as pd
df_hist=pd.DataFrame(history3.history)
df_hist.to_csv("E:\\Simulations\\Silvaco\\2D HEMT\\EDL\\DATA for modeling\\historyforVth.csv")

In [ ]:
df1=pd.DataFrame(list(zip(pred, act)), columns=['Pred', 'Act'])
df1.to_csv('act_pred_SS.csv')

In [49]:
import pandas as pd
df=pd.DataFrame(history3.history)

In [50]:
df.to_csv("E:\\Simulations\\Silvaco\\\\2D HEMT\\EDL\\DATA for modeling\\new_models_and_their_history\\L32_L32_L256_2D64_D3_46.csv")
model.save("E:\\Simulations\\Silvaco\\2D HEMT\\EDL\\DATA for modeling\\new_models_and_their_history\\L32_L32_L256_2D64_D3_46.h5")

In [ ]:
# for i in range(50):
r=model.predict(x_test[1,:])
#     print(y_test[i])
print('predicted: ', y_test[1],' actual: ', r )


In [ ]:
for i in range(len(X_test)):
    print(r[i]*1000,y_test[i]*1000)

In [ ]:
model = Sequential()
adam=optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_traindim[0].shape)))
model.add(Dropout(0.5))


model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer=adam, metrics=['MSE'])
model.summary()

In [ ]:
filepath="E:\\Simulations\\Silvaco\\2D HEMT\\To prepare paper 2 \\InAlGaN-AlN-GaN HEMT\\Machine Leaning Data\\lstmmodel.hdf5"
checkpointer = ModelCheckpoint( monitor='val_loss',filepath=filepath, verbose=1, save_best_only=True)
history=model.fit(x_traindim, y_traindim, validation_data=(x_testdim, y_testdim), epochs=100, batch_size=16, callbacks=[checkpointer])


In [ ]:
model.save("E:\\Simulations\\Silvaco\\2D HEMT\\EDL\\DATA for modeling\\fom23.37mse.h5")

In [ ]:
result= model.evaluate(x_testdim, y_testdim)

In [ ]:
print(result)

In [ ]:

for i in range(len(X_test)):
    print(r[i]*1000,y_test[i]*1000)

In [ ]:
import numpy as nd
xdim=np.expand_dims((X[42:83]), axis=1)
xdim.shape

In [ ]:
pred_id=model.predict(xdim)
pred_id

In [ ]:
for i in range(len(pred_id)):
    print(pred_id[i][0]*1000, y[42:83][i]*1000)

In [ ]:
X[0:41][2][0]

In [ ]:
a=[]
b=[]
c=[]
for i in range(len(pred_id)):
    a.append(pred_id[i][0])
    b.append(y[42:83][i])
    c.append(X[42:83][i][0])


In [ ]:
c

In [ ]:
import pandas as pd
q={'vd':c, 'pred_id':a, 'id':b }
df=pd.DataFrame(q)
df.to_csv('modelfile1.csv')

In [ ]:
pred_id=model.predict(x_testdim)
pred_id

In [ ]:
for i in range(len(pred_id)):
    print(pred_id[i][0]*1000 - y_testdim[i][0]*1000)